In [1]:
import os
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend\\research'

In [2]:
os.chdir('..')
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend'

In [3]:
from dataclasses import dataclass
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [5]:
data.isnull().sum() 

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [6]:

data.shape

(1599, 12)

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from box import ConfigBox

from src.datasci.entity.config_entity import DataIngestionCofig
from src.datasci.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH,
                                   SCHEMA_FILE_PATH)
from src.datasci.utils.common import create_directories, read_yaml


class ConfiguratonManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionCofig:
        config = self.config.data_ingestion #from yaml
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionCofig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    
    def get_data_validation_config(self)-> ConfigBox:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
            
        )
        return data_validation_config

In [10]:
import os
from src.datasci import logger

class DataValidation:
    def __init__(self, config:DataIngestionCofig):
        self.config = config
    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status: {validation_status}")
            logger.info(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e 
        

In [12]:
config = ConfiguratonManager()
data_validation_config = config.get_data_validation_config()   
data_validation = DataValidation(config=data_validation_config)
data_validation.validate_all_columns()

[2024-12-07 00:47:29,505: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-07 00:47:29,508: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-07 00:47:29,512: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-07 00:47:29,512: INFO: common: created directory at: artifacts]
[2024-12-07 00:47:29,525: INFO: common: created directory at: artifacts/data_validation]
[2024-12-07 00:47:29,544: INFO: 1988093747: Validation status: True]


True